In [5]:
import iris
from distributed import Client

c = Client('172.31.27.213:8786')
c

<Client: scheduler="172.31.27.213:8786" processes=1 cores=2>

In [21]:
def load_cubes(address):
    def fix_metadata(cube, field, filename):
        if not cube.coords('realization'):
            realization = int(filename.split('_')[-2])
            realization_coord = iris.coords.AuxCoord(
                realization, standard_name='realization', var_name='realization')
            cube.add_aux_coord(realization_coord)
        cube.coord('realization').points.dtype = 'int32'
        cube.coord('time').var_name = 'time'
        cube.coord('forecast_period').var_name = 'forecast_period'
    return iris.load(address, 'visibility_in_air', callback=fix_metadata)

print(load_cubes('./prods_op_mogreps-g_20160101_00_00_003.nc'))

0: visibility_in_air / (m)             (latitude: 600; longitude: 800)


/opt/conda/lib/python3.5/site-packages/iris/fileformats/cf.py:1140: IrisDeprecation: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warn_deprecated(msg)


In [122]:
from queue import Queue

input_q = Queue()

remote_q = c.scatter(input_q)

load_q = c.map(load_cubes, remote_q)

In [171]:
result_q = c.gather(load_q)

distributed.utils - ERROR - load_cubes-7fdc63dc56bf0c1afc7c3d6a4d92b80d
Traceback (most recent call last):
  File "/opt/conda/lib/python3.5/site-packages/distributed/client.py", line 894, in _gather
    six.reraise(type(st.exception),
AttributeError: exception

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.5/site-packages/distributed/utils.py", line 149, in f
    result[0] = yield gen.maybe_future(func(*args, **kwargs))
  File "/opt/conda/lib/python3.5/site-packages/tornado/gen.py", line 1015, in run
    value = future.result()
  File "/opt/conda/lib/python3.5/site-packages/tornado/concurrent.py", line 237, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 3, in raise_exc_info
  File "/opt/conda/lib/python3.5/site-packages/tornado/gen.py", line 1021, in run
    yielded = self.gen.throw(*exc_info)
  File "/opt/conda/lib/python3.5/site-packages/distributed/client.py", line 900, in 

In [124]:
from boto.s3.connection import S3Connection
import os

os.environ['S3_USE_SIGV4'] = 'True'

def load_data(q, bucket, prefix='prods'):
    conn = S3Connection(host='s3.eu-west-2.amazonaws.com')
    bucket = conn.get_bucket(bucket)
    results = []
    keys = iter(bucket.list(prefix=prefix))
    print(prefix)
    for k in keys:
        print(k.key)
        q.put('/usr/local/share/notebooks/data/mogreps-g/{}'.format(k.key))

In [125]:
load_data(input_q, 'mogreps-g', 'prods_op_mogreps-g_20160101_00')

prods_op_mogreps-g_20160101_00
prods_op_mogreps-g_20160101_00_00_003.nc
prods_op_mogreps-g_20160101_00_00_006.nc
prods_op_mogreps-g_20160101_00_00_009.nc
prods_op_mogreps-g_20160101_00_00_012.nc
prods_op_mogreps-g_20160101_00_00_015.nc
prods_op_mogreps-g_20160101_00_00_018.nc
prods_op_mogreps-g_20160101_00_00_021.nc
prods_op_mogreps-g_20160101_00_00_024.nc
prods_op_mogreps-g_20160101_00_00_027.nc
prods_op_mogreps-g_20160101_00_00_030.nc
prods_op_mogreps-g_20160101_00_00_033.nc
prods_op_mogreps-g_20160101_00_00_036.nc
prods_op_mogreps-g_20160101_00_00_039.nc
prods_op_mogreps-g_20160101_00_00_042.nc
prods_op_mogreps-g_20160101_00_00_045.nc
prods_op_mogreps-g_20160101_00_00_048.nc
prods_op_mogreps-g_20160101_00_00_051.nc
prods_op_mogreps-g_20160101_00_00_054.nc
prods_op_mogreps-g_20160101_00_00_057.nc
prods_op_mogreps-g_20160101_00_00_060.nc
prods_op_mogreps-g_20160101_00_00_063.nc
prods_op_mogreps-g_20160101_00_00_066.nc
prods_op_mogreps-g_20160101_00_00_069.nc
prods_op_mogreps-g_2016010

In [169]:
load_q.qsize()

692

In [170]:
result_q.qsize()

0

In [83]:
load_q.qsize()

690

In [84]:
remote_q.qsize()

0

In [118]:
load_q.get().result()

CancelledError: load_cubes-8f4bdb17afe2562cdf4027fb3b9a56d8

In [121]:
c.restart()

<Client: scheduler="172.31.18.5:8786" processes=60 cores=60>

In [175]:
load_q.get().result()

CancelledError: load_cubes-76125a261f16b23fb34a710718524264